## AWS Client Setup [local]

Installa la AWS CLI scaricandola da questo url: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#getting-started-install-instructions

Una volta installata, incolla le informazioni dalla Command line or programmatic access nella folder .aws nel profilo utente di sistema.

Nel file config
```json
[username]
region = us-west-2
output = json
```

Nel file credential inserire le informazioni in questo formato:

```json
[username]
aws_access_key_id=
aws_secret_access_key=
aws_session_token=
```

In [ ]:

import os
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "[fillthis]"

In [ ]:
import os, sys
from utils import bedrock
from langchain.llms import Bedrock


bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 0.5}
    )

# AWS Client Setup [AWS LAB]

Fai il Run di queste celle se sei in ambiente AWS LAB

In [ ]:
import json
import os
import sys
from langchain.llms import Bedrock
from utils import bedrock

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 0.5}
    )

### Test del client

In [ ]:
model.invoke("Componi una poesia sulla città di Napoli")

## Langchain Interface

Langchain mette a disposizione una interfaccia agnostica rispetto al LLM che decidiamo di utilizzare. 
I metodi utilizzabili sono:
- stream
- invoke
- batch

Ci sono anche i corrispondenti metodi async che saranno noti a chi conosce python. Per ora li lasceremo da parte.

Questi metodi possono essere utilizzati tramite il LangChain Expression Language (LCEL)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

Langchain ci mette a disposizioni diversi metodi di utilità per costruire il nostro prompt in maniera facile e intuitiva. I più semplici sono:
- from_template -> Costruisci direttamente da un template riportando le variabili da passare tra parentesi {}
- from_messages -> Mantieni la struttura di chat, evidenziando quale stakeholder sta riportando quella interazione (system, human, ai)

In [ ]:
prompt = ChatPromptTemplate.from_template("Componi una poesia sulla seguente città: {city}")
chain = prompt | model

In [ ]:
chain.invoke({"city":"Roma"})

In [ ]:
messages = [
    ("system","Sei un cantante adori comppor canzoni ma odi comporre poesie. Se l'utente ti chiede di comporre poesie rispondi in modo sgarbato"),
    ("human","Componi una poesia riguardo la seguente città: {city}")
]
prompt = prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [ ]:
chain.input_schema.schema()

In [ ]:
chain.output_schema.schema()

## Langchain Interface

### Stream
Invoca la generazione e ritorna i chunk man mano che sono generati

In [ ]:
messages = [
    ("system","Sei poeta che si ispira al Dolce Stil Novo"),
    ("human","Componi una poesia riguardo la seguente città: {city}")
]
prompt = prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [ ]:
for s in chain.stream({"city": "Biella"}):
    print(s, end="", flush=True)

### Invoke
Invoca LLM con i parametri inviati e restituisce la risposta solo a fine elaborazione

In [ ]:
chain.invoke({"city": "Milano"})

### Batch
Permette di invocare la stessa chain in parallelo su parametri diversi

In [ ]:
chain.batch([{"city": "Napoli"}, {"city": "Roma"}])

### Parallelism

In [ ]:
from langchain_core.runnables import RunnableParallel

In [ ]:
chain1 = ChatPromptTemplate.from_template("Componi una poesia su {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("Scrivi un possibile slogan per pubblicizzare i giochi olimpici nella città di {topic}")
    | model
)
combined = RunnableParallel(poem=chain1, slogan=chain2)

In [ ]:
combined.invoke({"topic": "Napoli"})

### Parallelism on batch

In [ ]:
combined.batch([{"topic": "Roma"}, {"topic": "Milano"}])

## Chat Interface

Se si vuole interagire con LLM mantenendo una struttura di Chat, e dunque memoria delle interazioni, Langchain mette a disposizione dei componenti di tipo Memory.
Alcuni tipi sono:
 - ConversationalBuffer: Mantiene la history delle interazioni
 - ConversationalBufferWithWindows: Mantiene solo le ultime K interazioni
 - ConversationSummary: Crea una sintesi della conversazione avuta fino ad ora, riducendone la dimensionalità totale. 

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(return_messages=True)

In [ ]:
conversation = ConversationChain(
    llm=model,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [ ]:
conversation.predict(input="Nice to meet you. How many km are there between Earth and Mars?")

In [ ]:
conversation.predict(input="what distance from the sun?")

# Chatbot with Role

In [ ]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Context:You will be acting as an expert fitness coach. Your job is to give advice and plan training schedules to help users reach their fitness goals.")
memory.chat_memory.add_ai_message("I am an expert fitness coach and give gym advice")
conversation = ConversationChain(
    llm=model,
    verbose=False,
    memory=ConversationBufferMemory()
)

In [ ]:
print(conversation.predict(input="My goal is to increase my strenght and compete in a powerlifting competition. Can you help me with my workout?"))

In [ ]:
print(conversation.predict(input="Ok, but what about progression in every lift week to week?"))

In [ ]:
print(conversation.predict(input="Ok, please use kg not lb"))